In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numba.cuda as cuda


device = cuda.get_current_device()

print("Device name:", device.name)
print("Device compute capability:", device.compute_capability)
print("Device threads per block:", device.WARP_SIZE)
print("Device number of multi-processors:", device.MULTIPROCESSOR_COUNT)

Device name: b'Tesla T4'
Device compute capability: (7, 5)
Device threads per block: 32
Device number of multi-processors: 40


In [ ]:
temp = []
import random
temp2 = []
with open('pp_tes.dat', 'r') as file:
    for line in file:
        for word in line.split():
            temp.append(word)
testing_data = [temp[i * 193:(i + 1) * 193] for i in range((len(temp) + 193 - 1) // 193)]
arrr = []
for i in testing_data:
    if(i[-1] == '0' or i[-1] == '1'):
        arrr.append(i)
random.shuffle(arrr)
testing_data = arrr[:50]
k=[]
for i in testing_data:

    i.extend(i)
    i.extend(i)
    i.extend(i)
    i.extend(i)
    i.extend(i)
    k.append(i)
testing_data=k

with open('pp_tra.dat', 'r') as file:
    for line in file:
        for word in line.split():
            temp2.append(word)
training_data = [temp2[i * 193:(i + 1) * 193] for i in range((len(temp2) + 193 - 1) // 193)]
arrr = []
for i in training_data:
    if(i[-1] == '0' or i[-1] == '1'):
        arrr.append(i)
random.shuffle(arrr)
training_data = arrr[:50]
k=[]
for i in training_data:
    i.extend(i)
    i.extend(i)
    i.extend(i)
    i.extend(i)
    i.extend(i)
    i.extend(i)
    k.append(i)
training_data=k
print(len(training_data[6]))

12352


# Parallel Implementation

In [ ]:
import numpy as np
import random
from math import pow
from numba import cuda
@cuda.jit
def distancebtwpoints_cuda(point1, point2, p, distance):
    distance_local = 0
    idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    if idx < len(point1):
        p1 = point1[idx]
        p2 = point2[idx]
        k = 1
        if p1 > p2:
            k = p1 - p2
        else:
            k = p2 - p1
        distance_local += k ** p
    cuda.atomic.add(distance, 0, distance_local)




def KNNC(testpoint, k, training, p):
    testpointfordist = testpoint[:-1]
    testpointfordist=np.array(testpointfordist).astype(np.float32)
    distanceofALL_totestpoint = []
    for trainpoint in training:
        trainpointfordist = trainpoint[:-1]
        trainpointfordist = np.float32(trainpointfordist)
        point1_gpu = cuda.to_device(trainpointfordist)
        distance = np.zeros(1, dtype=np.float32)
        distance_gpu = cuda.to_device(distance)
        point2_gpu = cuda.to_device(np.float32(testpointfordist))
        threadsperblock = 10
        blockspergrid = (len(trainpointfordist) + (threadsperblock - 1)) // threadsperblock
        distancebtwpoints_cuda[blockspergrid, threadsperblock](point1_gpu, point2_gpu, p, distance_gpu)
        distance = distance_gpu.copy_to_host()
        distancetuple = (distance[0], trainpoint[-1])
        distanceofALL_totestpoint.append(distancetuple)
    distanceofKnearest = sorted(distanceofALL_totestpoint)[:k]
    g = []
    for i in range(len(distanceofKnearest)):
        g.append(distanceofKnearest[i][1])
    res = max(set(g), key=g.count)
    return res
    cuda.select_device(0)
cuda.close()
cuda.select_device
cuda.select_device(0)



# Test KNN
# Test KNNC function




for i in testing_data:
    l = KNNC(i, 5, training_data, 10)
    print(l)

# serial implementation:

In [ ]:
import random
from math import pow

kaccuracies = []
temp = []
temp2 = []


### points===testing_data1{array}
def distancebtwpoints(point1, point2, p):
    distance = 0
    for i in range(len(point1)):
        p1 = float(point1[i])
        p2 = float(point2[i])
        k = abs(p1 - p2)
        distance += (k) ** p
    return pow(distance, 1 / p)


def KNNC(testpoint, k, training, p):  ##aray of arrays
#test point is an array
    testpointfordist = testpoint[:-1]
    distanceofALL_totestpoint = []
    for trainpoint in training:
        trainpointfordist = trainpoint[:-1]
        t = distancebtwpoints(trainpointfordist, testpointfordist, p)
        distancetuple = (t, trainpoint[-1])
        # print(distancetuple)
        distanceofALL_totestpoint.append(distancetuple)
    distanceofKnearest = sorted(distanceofALL_totestpoint)[:k]
    g = []
    for i in range(len(distanceofKnearest)):
        g.append(distanceofKnearest[i][1])
    res = max(set(g), key=g.count)
    print(res)
    return res

for i in testing_data:
  l=KNNC(i,5,training_data,10)
  print(l)
  
